# Step 1: Set up environment

#### To create and activate a virtual environment, run on your terminal:

**Windows:**


```
python -m venv venv
```
```
venv\Scripts\Activate
```

**macOS/Linux:**

```
python3 -m venv venv
```

```
source venv/bin/activate
```

#### After the environment is activated, install the requirements:

```
pip install -r requirements.txt
```


# Step 2: Pre-processing the document

#### To create and activate a virtual environment, run on your terminal:

For this tutorial, I am using a paper on Skin Cancer Detection using ML Techniques <sup>1</sup>,
that I have downloaded in my files. The same steps will apply for any paper / book just download it in pdf format and add your file path to the variable 'path'.  

<sup>1</sup> M. Vidya and M.V. Karki "Skin Cancer Detection using Machine Learning Techniques", 2020 IEEE International Conference on Electronics, Computing and Communication Technologies, Bangalore, India, 2020, pp. 1-5, doi 10.1109/CONECCT50063.2020.9198489.98489. 

